In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import requests
from datetime import datetime, timedelta
from tqdm import tqdm

In [37]:
def get_energy_data_json():

    # get all available time stamps
    stampsurl = "https://www.smard.de/app/chart_data/410/DE/index_quarterhour.json"
    response = requests.get(stampsurl)

    # ignore first 8 years (historic data is in already saved csv)
    timestamps = list(response.json()["timestamps"])[8*52 + 45:]

    col_names = ['timestamp_CET', 'gesamt']
    energydata = pd.DataFrame(columns=col_names)
    
    # loop over all available timestamps
    for stamp in tqdm(timestamps):

        dataurl = "https://www.smard.de/app/chart_data/410/DE/410_DE_quarterhour_" + str(stamp) + ".json"
        response = requests.get(dataurl)
        rawdata = response.json()["series"]

        for i in range(len(rawdata)):
            rawdata[i][0] = datetime.fromtimestamp(int(str(rawdata[i][0])[:10])).strftime("%Y-%m-%d %H:%M:%S")

        if energydata.empty:
            energydata = pd.DataFrame(rawdata, columns=col_names)
        else: 
            energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])

    energydata = energydata.dropna()
    energydata["timestamp_CET"] = pd.to_datetime(energydata.timestamp_CET).dt.tz_localize('CET', ambiguous='infer')
    energydata['timestamp_UTC'] = energydata['timestamp_CET'].dt.tz_convert('UTC')
    
    # set UTC date_time as index
    energydata.set_index("timestamp_UTC", inplace=True)

    # resample
    # energydata = energydata.resample("1h", label="left").sum()

    return energydata

In [43]:
get_energy_data_json()

100%|██████████| 12/12 [00:05<00:00,  2.32it/s]


,timestamp_CET,gesamt
timestamp_UTC,,
2023-10-29 23:00:00+00:00,2023-10-30 00:00:00+01:00,10971.75
2023-10-29 23:15:00+00:00,2023-10-30 00:15:00+01:00,10798.75
2023-10-29 23:30:00+00:00,2023-10-30 00:30:00+01:00,10732.50
2023-10-29 23:45:00+00:00,2023-10-30 00:45:00+01:00,10690.00
2023-10-30 00:00:00+00:00,2023-10-30 01:00:00+01:00,10609.75
...,...,...
2024-01-18 12:45:00+00:00,2024-01-18 13:45:00+01:00,17500.25
2024-01-18 13:00:00+00:00,2024-01-18 14:00:00+01:00,17451.00
2024-01-18 13:15:00+00:00,2024-01-18 14:15:00+01:00,17366.25


In [103]:
def get_energy_data_today(to_date=None, recycle=False) :

    os.chdir("C:/2023_11-PTSFC")
    # os.chdir("../2023_11-PTSFC")
    print(f"> cwd = {os.getcwd()}")

    # if input param to_date is not given, use today's date
    if to_date is None :
        to_date = datetime.today().strftime("%Y%m%d")

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # check if file already exists
        
    # if file DNE then see if a later date exists 
    # which we can also take but just need to cut off the irrelevant tail

    # check all files that start with "2015-01-01_" and end with "_energy.csv"
    # and take the first one with a date later than to_date
    # if no such file exists then proceed download data from SMARD
        
    if recycle :

        to_date_fname = datetime.strptime(to_date, "%Y%m%d").strftime("%Y-%m-%d")
        fname = f"2015-01-01_{to_date_fname}_energy.csv"
        print(f"> checking if {fname} or later already exists ...")

        if os.path.isfile(f"./data/{fname}"):
            print(f"> {fname} already exists, reading from csv ...")
            df = pd.read_csv(f"./data/{fname}")
            return df

        else :
            files = os.listdir("./data")
            files = [f for f in files if f.startswith("2015-01-01_")]
            files = [f for f in files if f.endswith("_energy.csv")]
            files = [f for f in files if f > fname]

            # take last file in list
            if len(files) > 0 :             
                recent_fname = files[-1]
                print(f"using {recent_fname} instead of {fname} !")

                df = pd.read_csv(f"./data/{recent_fname}")
                cutoff = (datetime.strptime(to_date, "%Y%m%d") + timedelta(days=1)).strftime("%Y-%m-%d")
                df = df[df['timestamp_CET'] < cutoff]
                return df

            else :
                print(f"{fname} does not exist, downloading data from SMARD !")
    
    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
    
    # if to_date is earlier than 2023_11_01, use historic data only
    hist_fname = f"Realisierter_Stromverbrauch_201501010000_202310312359_Viertelstunde.csv"
    hist_df = pd.read_csv(f"./data/{hist_fname}", sep=";", decimal=",")
    # Rename columns for convenience and remove cols = ['residual', 'pump']
    hist_df.columns = ["datum", "anfang", "ende", "gesamt", "residual", "pump"]
    hist_df.drop(columns=['residual', 'pump'], inplace=True)
    # replace '-' with NA
    hist_df.replace('-', pd.NA, inplace=True)

    # Merge date and time column and set that as the index
    hist_df["timestamp"] = pd.to_datetime(hist_df['datum'] + ' ' + hist_df['anfang'], format='%d.%m.%Y %H:%M')
    # change datetime to utc time
    hist_df['timestamp_CET'] = hist_df['timestamp'].dt.tz_localize('CET', ambiguous='infer')
    hist_df['timestamp_UTC'] = hist_df['timestamp_CET'].dt.tz_convert('UTC')
    hist_df.set_index("timestamp_UTC", inplace=True)
    # make index datetime object
    hist_df.index = pd.to_datetime(hist_df.index)
    # drop redundant columns
    hist_df = hist_df[['timestamp_CET', 'gesamt']].copy()

    # Replace "." with "" and then replace "," with "."
    hist_df[['gesamt']] = hist_df[['gesamt']].apply(lambda x: x.str.replace('.', '', regex=False))
    hist_df[['gesamt']] = hist_df[['gesamt']].apply(lambda x: x.str.replace(',', '.', regex=False))
    hist_df[['gesamt']] = hist_df[['gesamt']].apply(pd.to_numeric)

    if to_date > "20231101" :
        print(f"> to_date is later than 2023-11-01, using recent data as well !")
        recent_df = get_energy_data_json()
        # get last index of hist_df
        last_index = hist_df.index[-1]
        # drop rows of recent_df that are already in hist_df
        recent_df = recent_df.loc[last_index + timedelta(minutes=15):]
        # merge 2 dataframes
        df_utc = pd.concat([hist_df, recent_df])

    else :
        print(f"> to_date is earlier than 2023-11-01, using historic data only !")
        df_utc = hist_df.copy()

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # cutoff date
    cutoff = (datetime.strptime(to_date, "%Y%m%d") + timedelta(days=1)).strftime("%Y-%m-%d")
    df_utc = df_utc[df_utc['timestamp_CET'] < cutoff]

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # if there are consecutive trailing NaNs, drop them ... find last valid index
    last_valid_index = df_utc['gesamt'].last_valid_index()

    if last_valid_index is not None:
        df_utc = df_utc.loc[:last_valid_index]

    print(f"> {df_utc['gesamt'].isna().sum()} NA in df")
    print(f"> last valid index = {last_valid_index}")

    # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
    # Interpolate missing values in between
    df_utc_interp = df_utc.copy()
    df_utc_interp['gesamt'].interpolate(method='time', inplace=True)

    # resample at hourly level
    df_hourly = df_utc_interp.resample("1h", label="left").agg({'gesamt':'sum','timestamp_CET':'first'})

    # reorder columns
    df_hourly = df_hourly[["timestamp_CET", "gesamt"]]

    # save to csv
    to_date_str = last_valid_index.strftime("%Y-%m-%d")
    fname = f"2015-01-01_{to_date_str}_energy"
    df_hourly.to_csv(f"./data/{fname}.csv")
    print(f"> done and saved to {fname}.csv")

    return df_hourly

In [104]:
df_energy = get_energy_data_today(to_date="20240114")
df_energy.tail()

> cwd = C:\2023_11-PTSFC
> to_date is later than 2023-11-01, using recent data as well !


100%|██████████| 12/12 [00:04<00:00,  2.60it/s]


> 0 NA in df
> last valid index = 2024-01-14 22:45:00+00:00
> done and saved to 2015-01-01_2024-01-14_energy.csv


,timestamp_CET,gesamt
timestamp_UTC,,
2024-01-14 18:00:00+00:00,2024-01-14 19:00:00+01:00,60132.75
2024-01-14 19:00:00+00:00,2024-01-14 20:00:00+01:00,58025.75
2024-01-14 20:00:00+00:00,2024-01-14 21:00:00+01:00,56493.75
2024-01-14 21:00:00+00:00,2024-01-14 22:00:00+01:00,55950.50
2024-01-14 22:00:00+00:00,2024-01-14 23:00:00+01:00,54027.50
